# Observatories

This notebook queries and visualizes information about EMO-BON observatories from the knowledge graph.

In [ ]:
from pysema import SPARQLConnection
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
# Connect to EMO-BON Knowledge Graph
graphdb_url = "https://emobon-kb.vliz.be/repositories/kgap"
conn = SPARQLConnection(graphdb_url)

## List All Observatories

This query retrieves all observatories in the EMO-BON network.

In [ ]:
query_observatories = """
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX emobon: <http://www.embrc.eu/emobon/EmobonOntology#>
PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?observatory ?name ?location
WHERE {
  ?observatory a emobon:Observatory .
  OPTIONAL { ?observatory schema:name ?name . }
  OPTIONAL { ?observatory schema:location ?location . }
}
ORDER BY ?name
"""

observatories_df = conn.query(query_observatories)
print(f"Found {len(observatories_df)} observatories")
observatories_df.head(10)

## Observatories by Country

This query groups observatories by their country.

In [ ]:
query_by_country = """
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX emobon: <http://www.embrc.eu/emobon/EmobonOntology#>
PREFIX schema: <http://schema.org/>

SELECT ?country (COUNT(?observatory) as ?count)
WHERE {
  ?observatory a emobon:Observatory .
  OPTIONAL { ?observatory schema:addressCountry ?country . }
}
GROUP BY ?country
ORDER BY DESC(?count)
"""

country_df = conn.query(query_by_country)

if not country_df.empty:
    fig = px.bar(country_df, x='country', y='count', 
                 title='Number of Observatories by Country',
                 labels={'country': 'Country', 'count': 'Number of Observatories'},
                 template='plotly_white')
    fig.update_traces(marker_color='#1f77b4')
    fig.show()
else:
    print("No country data available")

## Observatory Details

Get detailed information about each observatory including sampling features.

In [ ]:
query_details = """
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX emobon: <http://www.embrc.eu/emobon/EmobonOntology#>
PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?observatory ?name ?description (COUNT(?feature) as ?num_features)
WHERE {
  ?observatory a emobon:Observatory .
  OPTIONAL { ?observatory schema:name ?name . }
  OPTIONAL { ?observatory schema:description ?description . }
  OPTIONAL { ?observatory sosa:hasSamplingFeature ?feature . }
}
GROUP BY ?observatory ?name ?description
ORDER BY DESC(?num_features)
LIMIT 20
"""

details_df = conn.query(query_details)
details_df.head(10)